# Anime Recommendation System: Introduction

## Objective
The goal of this project is to build a **recommendation system** that can suggest anime to users based on their past ratings and anime characteristics.

## Dataset Overview
We are using the [Anime Dataset](#https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database?select=rating.csv) containing:
- **Anime information**: anime_id,name,genre,type,episodes,rating,members.
- **User ratings**: user_id,anime_id,rating.

## Project Goals
1. Explore and preprocess the dataset to handle sparsity, missing values and noisy ratings.
2. Implement multiple recommendation techniques:
    - **Baseline recommendation**: Simple popularity-based recommendations.
    - **Collaborative Filtering (CF)**: Suggest anime based on user-user or item-item similarity.
    - **Content-Based Filtering (CBF)**: Recommend anime using genre, type, and other features.
    - **Hybrid approaches**: Combine CF and CBF for better predictions.
3. Evaluate model performance using metrics like **RMSE** and **MAE**.
4. Visualise recommendations and analyze patterns in user preferences and anime clusters.

## Why This Matters
Recommendation systems help users **discover content they are likely to enjoy** and are a critical part of many modern applications. By combining user behavior and content features, I aim to build a system that can provide **personalized anime recommendations** and gain insights into anime clustering and user preferences.

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully!")

Libraries loaded successfully!


# Data Preprocessing

Before building any recommendation system, we need to **clean, preprocess and structure the data**. This involves:

1. **Handling missing values**. 
2. **Filtering invalid ratings**.  
4. **Creating a user-item rating matrix**.

In [2]:
# Loading datasets

anime_df = pd.read_csv('data/anime.csv')
ratings_df = pd.read_csv('data/rating.csv')

print(f"Anime dataset: {anime_df.shape}")
print(f"Ratings dataset: {ratings_df.shape}")

# Displaying basic information

print(f"Unique anime: {anime_df['anime_id'].nunique()}")
print(f"Unique users: {ratings_df['user_id'].nunique()}")
print(f"Total ratings: {ratings_df['rating'].value_counts()}")

# Display data 

print(anime_df.head())
print(ratings_df.head())

Anime dataset: (12294, 7)
Ratings dataset: (7813737, 3)
Unique anime: 12294
Unique users: 73515
Total ratings: rating
 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: count, dtype: int64
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17

In [3]:
# Cleaning anime data

anime_df['rating'] = pd.to_numeric(anime_df['rating'], errors='coerce')
anime_df['episodes'] = pd.to_numeric(anime_df['episodes'], errors='coerce')
anime_df['members'] = pd.to_numeric(anime_df['members'], errors='coerce')

# removing all anime without ratings (-1) and adult content

anime_clean = anime_df.dropna(subset=['rating'].copy())
anime_clean = anime_clean[~anime_clean['genre'].str.contains('Hentai', case=False, na=False)].copy()
print(f"Anime entries after cleaning: {len(anime_clean)}")

# Cleaning rating data

ratings_clean = ratings_df[(ratings_df['rating'] != -1) & (ratings_df['rating'].notna())].copy()
print(f"Ratings after cleaning: {len(ratings_clean)}")

# Removing all users and items with few interactions (deals with the sparcity)

user_counts = ratings_clean['user_id'].value_counts()
item_counts = ratings_clean['anime_id'].value_counts()

# Keeping anime with at least 5 ratings

min_user_ratings = 5
min_item_ratings = 5

valid_users = user_counts[user_counts >= min_user_ratings].index
valid_items = item_counts[item_counts >= min_item_ratings].index

ratings_filtered = ratings_clean[
    (ratings_clean['user_id'].isin(valid_users)) & 
    (ratings_clean['anime_id'].isin(valid_items))
].copy()

print(ratings_filtered.head())

# Calculating sparcity

n_users = ratings_filtered['user_id'].nunique()
n_items = ratings_filtered['anime_id'].nunique()
n_ratings = len(ratings_filtered)
sparsity = (1 - n_ratings / (n_users * n_items)) * 100

print(f"Users: {n_users:,}")
print(f"Items: {n_items:,}")
print(f"Ratings: {n_ratings:,}")
print(f"Sparsity: {sparsity:.2f}%")
print(f"Avg ratings per user: {n_ratings/n_users:.1f}")
print(f"Avg ratings per item: {n_ratings/n_items:.1f}")



Anime entries after cleaning: 10931
Ratings after cleaning: 6337241
     user_id  anime_id  rating
156        3        20       8
157        3       154       6
158        3       170       9
159        3       199      10
160        3       225       9
Users: 60,970
Items: 8,030
Ratings: 6,314,650
Sparsity: 98.71%
Avg ratings per user: 103.6
Avg ratings per item: 786.4


## Popularity Baseline Recommender

The first step in building our recommendation system, we implement a **popularity-based baseline**. This baseline recommends the most popular items to all users, ignoring personal preferences. 

### How it Works:
1. **Fit Phase**:
- Compute average rating and count for each item.
- Filter items with too few ratings (to reduce noise).
- Rank items by average rating to determine popularity.
2. **Prediction**:
- If the item exists in the popular list, return its average rating.
- Otherwise, return the global average rating.
3. **Recommendation**:
- Return top-N popular items.
- Optionally, exclude items the user has already seen.

In [4]:
class PopularityBaseline:

    def __init__(self):

        self.popular_items = None
        self.global_mean = None

    def fit(self, ratings_df):
        # Learns the most popular anime from the data

        # Calculate the popularity score

        item_stats = ratings_df.groupby('anime_id').agg({'rating' : ['mean','count']}).round(3)
        item_stats.columns = ['avg_rating','rating_count']

        # Only get anime with more than 10 ratings

        item_stats = item_stats[item_stats['rating_count'] >= 10]

        # Sort the anime by their avg_rating
        self.popular_items = item_stats.sort_values('avg_rating', ascending = False)
        self.global_mean = ratings_df['rating'].mean()

        print(f"Baseline trained on {len(item_stats)} popular items")
        return self

    def predict(self, user_id, item_id):
        # Predicts the rating for a user-item pair
        if self.popular_items is None:
            return self.global_mean
        
        if item_id in self.popular_items.index:
            return self.popular_items.loc[item_id, 'avg_rating']
        else:
            return self.global_mean
        
    def recommend(self, user_id, n_recommendations, exclude_seen = None):
        # Recomend the top popular items

        recommendations = self.popular_items.head(n_recommendations).index.tolist()

        if exclude_seen:
            # Get all the popular items which have not been seen by the user
            recommendations = [item for item in recommendations if item not in exclude_seen]

            remaining = n_recommendations - len(recommendations)
            if remaining > 0:
                additional = self.popular_items.iloc[n_recommendations:n_recommendations+remaining*2]
                for item in additional.index:
                    if item not in exclude_seen:
                        recommendations.append(item)
                        if len(recommendations) >= n_recommendations:
                            break

            return recommendations[:n_recommendations]

In [5]:
# Train baseline model
print("Training Popularity Baseline")
baseline_model = PopularityBaseline()
baseline_model.fit(ratings_filtered)

# Show the top recommendation

top_anime_ids = baseline_model.popular_items.head(10).index
for i, anime_id in enumerate(top_anime_ids, 1):
    anime_name = anime_clean[anime_clean['anime_id'] == anime_id]['name'].iloc[0]
    avg_rating = baseline_model.popular_items.loc[anime_id, 'avg_rating']
    rating_count = baseline_model.popular_items.loc[anime_id, 'rating_count']
    print(f'{1:2d}. {anime_name[:40]:40} | Rating: {avg_rating:.2f} |Count: {rating_count:,}')

Training Popularity Baseline
Baseline trained on 7364 popular items
 1. Gintama°                                 | Rating: 9.45 |Count: 1,182
 1. Kimi no Na wa.                           | Rating: 9.42 |Count: 1,948
 1. Ginga Eiyuu Densetsu                     | Rating: 9.39 |Count: 799
 1. Fullmetal Alchemist: Brotherhood         | Rating: 9.32 |Count: 21,220
 1. Gintama&#039;                            | Rating: 9.27 |Count: 3,098
 1. Steins;Gate                              | Rating: 9.26 |Count: 17,019
 1. Hunter x Hunter (2011)                   | Rating: 9.23 |Count: 7,418
 1. Gintama                                  | Rating: 9.23 |Count: 4,222
 1. Gintama&#039;: Enchousen                 | Rating: 9.20 |Count: 2,121
 1. Gintama Movie: Kanketsu-hen - Yorozuya y | Rating: 9.19 |Count: 2,139


## Content-Based Filtering for Anime Recommendations

This section implements a **content-based recommendation system** that leverages **anime features** to generate personalised recommendations.

### Key Steps:

1. **Feature Preparation**
   - Genre features transformed using **TF-IDF**.
   - Anime type features encoded as **one-hot vectors**.
   - Numeric features like `rating`, `episodes`, and `members` **normalised**.
   - Combined all features into a **single feature matrix**.

2. **Similarity Computation**
   - Compute **cosine similarity** between all anime.
   - Store similarity matrix.

3. **Personalized Recommendation**
   - For each anime a user has rated, retrieve similar anime.
   - Weight similarity scores by the user’s rating.
   - Aggregate scores and sort to recommend the top-N anime.

In [6]:
class ContentBasedFilter:

    def __init__(self):
        self.item_features = None
        self.item_similarity_matrix = None
        self.tfidf_vectorizer = None
        self.anime_to_idx = None
        self.idx_to_anime = None

    def prepare_features(self, anime_df):
        # preparing the feature matrix for anime


        anime_ids = anime_df['anime_id'].values
        self.anime_to_idx = {anime_id: idx for idx, anime_id in enumerate(anime_ids)}
        self.idx_to_anime = {idx : anime_id for anime_id, idx in self.anime_to_idx.items()}

        features_list = []

        # Genres features using TF-IDF

        genres = anime_df['genre'].fillna('Unknown')
        self.tfidf_vectorizer = TfidfVectorizer(max_features=50, stop_words=None)
        genre_features = self.tfidf_vectorizer.fit_transform(genres).toarray()
        features_list.append(genre_features)

        # Type features

        type_dummies = pd.get_dummies(anime_df['type']).values
        features_list.append(type_dummies)

        # Numerical features (normalised)

        numerical_features = anime_df[['rating', 'episodes', 'members']].copy()
        numerical_features['episodes'] = numerical_features['episodes'].fillna(1)

        # normalise numerical features 
        for col in numerical_features.columns:
            numerical_features[col] = (numerical_features[col] - numerical_features[col].mean()) / numerical_features[col].std()

        features_list.append(numerical_features.values)

        # Combine all features

        self.item_features = np.hstack(features_list)

        print(f"Content features prepared: {self.item_features.shape}")
        return self
    
    def fit(self, anime_df):
        # Train the content-based model

        self.prepare_features(anime_df)

        # Calculate the item similarity matrix

        print("Calculating item simularity matrix")
        self.item_similarity_matrix = cosine_similarity(self.item_features)

        print("Training complete")
        print(f"Feature dimensions: {self.item_features.shape[1]}")
        print(f"Items: {self.item_features.shape[0]}")

        return self
    

    def get_silmilar_items(self, anime_id, n_similar=50):
        # Get the most similar items to the anime given

        if anime_id not in self.anime_to_idx:
            return []
        
        item_idx = self.anime_to_idx[anime_id]
        similarities = self.item_similarity_matrix[item_idx]

        # Get indicies of the most similar items (except itself)

        similar_indices = np.argsort(similarities)[::-1][1:n_similar+1]
        similar_anime_ids = [self.idx_to_anime[idx] for idx in similar_indices]
        similarity_scores = similarities[similar_indices]

        return list(zip(similar_anime_ids, similarity_scores))
    
    def recommend(self, user_ratings, n_recommendations = 10):
        # Recommend item based on the user's rating history
        # user_ratings : dict {anime_id, rating}

        if not user_ratings:
            return []
        
        item_scores = defaultdict(float)
        item_weights = defaultdict(float)

        for rated_anime_id, rating in user_ratings.items():
            if rated_anime_id not in self.anime_to_idx:
                continue

                # Get similar items to the given anime

            similar_items = self.get_silmilar_items(rated_anime_id, n_similar=50)

            for similar_anime_id, similarity in similar_items:
                if similar_anime_id not in user_ratings: # Avoid recommending watched anime
                    # Weight similarity by user's rating (higher rated items get more weight)
                    weight = (rating - 5) * similarity 
                    item_scores[similar_anime_id] += item_weights
                    item_weights[similar_anime_id] += abs(similarity)

            # Calculate the final scores
            final_scores = {}
            for anime_id in item_scores:
                if item_weights[anime_id] > 0:
                       final_scores[anime_id] = item_scores[anime_id]/ item_weights[anime_id]

            # Sort the final scores
            recommendations = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
            return [anime_id for anime_id, score in recommendations[:n_recommendations]]


In [7]:
# Training content-based filter
print("Training content-based filter")
content_filter = ContentBasedFilter()
content_filter.fit(anime_clean)

# Example: find an anime similar to a popular one
sample_anime_id = anime_clean.iloc[486]['anime_id']
sample_anime_name = anime_clean.iloc[486]['name']

print(f"Finding similar anime to {sample_anime_name}")
similar_anime = content_filter.get_silmilar_items(sample_anime_id, n_similar=5)

for i, (anime_id, similarity) in enumerate(similar_anime, 1):
    anime_name = anime_clean[anime_clean['anime_id'] == anime_id]['name']
    if not anime_name.empty:
        print(f"{i}. {anime_name.iloc[0][:40]:40} | Similarity {similarity:.3f}")

Training content-based filter
Content features prepared: (10931, 56)
Calculating item simularity matrix
Training complete
Feature dimensions: 56
Items: 10931
Finding similar anime to Boruto: Naruto the Movie
1. The Last: Naruto the Movie               | Similarity 0.982
2. Naruto: Shippuuden Movie 6 - Road to Nin | Similarity 0.982
3. Naruto: Shippuuden Movie 4 - The Lost To | Similarity 0.976
4. Naruto: Shippuuden Movie 3 - Hi no Ishi  | Similarity 0.975
5. Dragon Ball Z Movie 14: Kami to Kami     | Similarity 0.972


# User-Based Collaborative Filtering

Now that we have popularity and content-based approaches, we implement **Collaborative Filtering (CF)**, 
which leverages similarities between users to generate recommendations. 

### Key Steps:
1. **User-Item Matrix Creation**  
   - Construct a sparse matrix where each row corresponds to a user and each column corresponds to an anime.  
   - Matrix entries represent ratings (0 if unrated).  
   - Also generate index mappings for users and items.  

2. **User-Based CF Algorithm**  
   - Calculate similarity between users using **cosine similarity** on their overlapping ratings.  
   - Predict ratings for unseen anime based on ratings from similar users.  
   - Use weighted averages where more similar users have greater influence.  

3. **Model Training & Recommendations**  
   - Fit the CF model on the user-item matrix.  
   - For a given user, recommend anime they haven’t rated yet by predicting which ones they would likely rate highly.  
   - Demonstrate recommendations for a sample user.  


In [8]:
def create_user_item_matrix(rating_df):
    # Creates a spase user_item rating matrix

    # Creat mappings for matrix indices

    user_ids = sorted(ratings_df['user_id'].unique())
    item_ids = sorted(ratings_df['anime_id'].unique())

    user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_to_idx = {item_id: idx for idx, item_id in enumerate(item_ids)}

    # Create matrix

    n_users, n_items = len(user_ids), len(item_ids)
    matrix = np.zeros((n_users, n_items))

    for _, row in ratings_df.iterrows():
        user_idx = user_to_idx[row['user_id']]
        item_idx = item_to_idx[row['anime_id']]
        matrix[user_idx, item_idx] = row['rating']

    return matrix, user_to_idx, item_to_idx, user_ids, item_ids

In [9]:
# Creating sample matrix
print('Creating user-item matrix')

sample_size = 10000
ratings_sample = ratings_filtered.sample(n=min(sample_size, len(ratings_filtered)), random_state=42)
user_item_matrix, user_to_index, item_to_index, user_ids, item_ids = create_user_item_matrix(ratings_sample)

print(f"User-item matrix shape: {user_item_matrix.shape}")
print(f"Non-zero entries: {np.count_nonzero(user_item_matrix):,}")
print(f"Sparsity: {(1 - np.count_nonzero(user_item_matrix) / user_item_matrix.size) * 100:.2f}%")

Creating user-item matrix
User-item matrix shape: (73515, 11200)
Non-zero entries: 7,813,730
Sparsity: 99.05%


In [10]:
class UserBasedCF:
    # User-based collaborative filtering

    def __init__(self, k = 20, similarity_threshold = 0.1):
        self.k = k # number of similar users to consider
        self.similarity_threshold = similarity_threshold
        self.user_item_matrix = None
        self.user_similarities = None
        self.user_means = None

    def fit(self, user_item_matrix):
        # Train the model on user-item matrix

        self.user_item_matrix = user_item_matrix

        # Calculate the user means (for mean-centered ratings)

        self.user_means = np.array([user_ratings[user_ratings > 0].mean() if np.any(user_ratings > 0) else 0 for user_ratings in user_item_matrix])

        print("User-based CF model trained")
        print(f"Matrix shape: {user_item_matrix.shape}")
        print(f"Using k={self.k} similar users")

    def get_user_similarity(self, user1_idx, user2_idx):
        # Calculate cosine similarity between two users

        user1_ratings = self.user_item_matrix[user1_idx]
        user2_ratings = self.user_item_matrix[user2_idx]

        # Find common rated items 

        common_items = (user1_ratings > 0) & (user2_ratings > 0)

        if np.sum(common_items) < 2:
            return 0
        
        user1_common = user1_ratings[common_items]
        user2_common = user2_ratings[common_items]

        # Calculate cosine simularity

        dot_product = np.dot(user1_common, user2_common)
        norm1 = np.linalg.norm(user1_common)
        norm2 = np.linalg.norm(user2_common)

        if norm1 == 0 or norm2 == 0:
            return 0
        
        return dot_product / (norm1 * norm2)
    

    def predict(self, user_idx, item_idx):
        # Predict rating for the user-item pairs

        if self.user_item_matrix[user_idx, item_idx] > 0:
            # User has already rated this anime
            return self.user_item_matrix[user_idx, item_idx]
        
        similarites = []
        ratings = []

        for other_user_idx in range(len(self.user_item_matrix)):
            if other_user_idx == user_idx:
                continue

            if self.user_item_matrix[other_user_idx, item_idx] > 0:
                sim = self.get_user_similarity(user_idx, other_user_idx)
                if sim > self.similarity_threshold:
                    similarites.append(sim)
                    ratings.append(self.user_item_matrix[other_user_idx,item_idx])

        if not similarites:
            return self.user_means[user_idx] if self.user_means[user_idx] > 0 else 3.0
        
        # Wighted average prediction

        similarites = np.array(similarites)
        ratings = np.array(ratings)

        if np.sum(similarites) == 0:
            return self.user_means[user_idx] if self.user_means[user_idx] > 0 else 3.0
        
        predicted_rating = np.sum(similarites * ratings) / np.sum(similarites)
        return np.clip(predicted_rating, 1, 10) # Ensure the rating is in a valid range
    
    def recommend(self, user_idx, n_recommendations=10, exclude_seen=True):
        # Recommend item for a user

        user_ratings = self.user_item_matrix[user_idx]
        unrated_items = np.where(user_ratings == 0)[0]

        if len(unrated_items) == 0:
            return []
        
        # Predict ratings for unrated items

        predictions = []
        for item_idx in unrated_items:
            pred_rating = self.predict(user_idx, item_idx)
            predictions.append((item_idx, pred_rating))

        # Sort by predicted rating and return top N

        predictions.sort(key=lambda x: x[1], reverse=True)
        return [item_idx for item_idx, _ in predictions[:n_recommendations]]

In [11]:
# Training user-based CF
print("Training user-based collaborative filtering")
user_cf = UserBasedCF(k=20)
user_cf.fit(user_item_matrix)

# Example recommendation

test_user_idx = 0
if test_user_idx < len(user_item_matrix):
    print(f"Sample recommendations for User : {user_ids[test_user_idx]}:")
    recommendations = user_cf.recommend(test_user_idx, n_recommendations=5)

    for i, item_idx in enumerate(recommendations,1):
        item_id = item_ids[item_idx]
        anime_name = anime_clean[anime_clean["anime_id"] == item_id]['name']
        if not anime_name.empty:
            predicted_rating = user_cf.predict(test_user_idx, item_idx)
            print(f"{i}. {anime_name.iloc[0][:40]:40} | Predicted Rating: {predicted_rating:.2f}")

Training user-based collaborative filtering
User-based CF model trained
Matrix shape: (73515, 11200)
Using k=20 similar users
Sample recommendations for User : 1:
1. Nurse Angel Ririka SOS                   | Predicted Rating: 10.00
2. Kingyo Chuuihou!                         | Predicted Rating: 10.00
3. Oishinbo                                 | Predicted Rating: 10.00
4. Future GPX Cyber Formula: Early Days Ren | Predicted Rating: 10.00
5. PostPet Momobin                          | Predicted Rating: 10.00


## Hybrid Recommendation System – Matrix Factorisation (SVD)

Matrix Factorisation is a latent-factor approach that reduces the high-dimensional user–item rating matrix into a smaller set of **hidden features** that capture patterns in user preferences and item attributes.

### Key Idea
Instead of relying only on observed similarities (like CF) or metadata (like content-based), this method learns a **compressed representation** of both users and items:
- **Users** are mapped into a latent feature space based on their rating behavior.
- **Items** are mapped into the same latent space based on who rated them and how.
- The interaction between a user’s vector and an item’s vector predicts ratings.

### Steps in Implementation
1. **Global Mean Centering**  
   - Calculate the average rating across the dataset.  
   - Subtract this mean from observed ratings to focus on relative preferences.

2. **Truncated SVD**  
   - Decompose the user–item matrix into:  
     - `user_factors` → low-dimensional user representations.  
     - `item_factors` → low-dimensional item representations.  

3. **Prediction**  
   - For user *u* and item *i*, the rating is estimated as:  
     \[
     \hat{r}_{ui} = \mu + U_u \cdot V_i
     \]  
     Where \(\mu\) is the global mean rating, and \(U_u, V_i\) are latent vectors.

4. **Recommendation**  
   - Predict scores for all unrated items.  
   - Rank them and recommend the top-N highest scoring items.



In [12]:
class MatrixFactorisationSVD:
    # Matrix Factorisation using Truncated SVD

    def __init__(self, n_components = 50):
        self.n_components = n_components
        self.svd = None
        self.user_factors = None
        self.item_factors = None
        self.global_mean = None

    def fit(self, user_item_matrix):
        # Training the SVD model 

        print(f"Training matrix factorisation (SVD) with {self.n_components} components")

        self.global_mean = user_item_matrix[user_item_matrix > 0].mean()

        # Center centered matrix (only for non-zero entries)

        centered_matrix = user_item_matrix.copy()
        mask = user_item_matrix > 0
        centered_matrix[mask] -= self.global_mean

        # Apply SVD

        self.svd = TruncatedSVD(n_components=self.n_components, random_state=42)
        self.user_factors = self.svd.fit_transform(centered_matrix)

        # Get item factors

        self.item_factors = self.svd.components_.T

        print(f"SVD model trained")
        print(f"Explained variance ratio: {self.svd.explained_variance_ratio_.sum():.3f}")
        
        return self
    
    def predict(self, user_idx, item_idx):
        # Predict the rating for user-item pairs

        prediction = self.global_mean + np.dot(self.user_factors[user_idx], self.item_factors[item_idx])
        return np.clip(prediction, 1, 10)
    
    def recommend(self, user_idx, n_recommendations=10, exclude_seen=True):
        # Recommend items for a user
        # Calculate predictions for all items

        user_vector = self.user_factors[user_idx]
        predictions = self.global_mean + np.dot(self.item_factors, user_vector)

        # Gret indices of top predictions

        top_indices = np.argsort(predictions)[::-1]

        recommendations = []

        for item_idx in top_indices:
            if len(recommendations) >= n_recommendations:
                break

            # Exclude items the user has already seen
            if exclude_seen and user_item_matrix[user_idx, item_idx] > 0:
                continue

            recommendations.append(item_idx)

        return recommendations

In [13]:
# Training SVD model 

print("Training Matrix Factorisation Mode")
svd_model = MatrixFactorisationSVD(n_components=50)
svd_model.fit(user_item_matrix)

# Example recommendations

if test_user_idx < len(user_item_matrix):
    print(f"SVD Recommendation for User {user_ids[test_user_idx]}:")
    svd_recommendations = svd_model.recommend(test_user_idx, n_recommendations = 5)
    for i, item_idx in enumerate(svd_recommendations, 1):
        item_id = item_ids[item_idx]
        anime_name = anime_clean[anime_clean['anime_id'] == item_id]['name']
        if not anime_name.empty:
            predicted_rating = svd_model.predict(test_user_idx, item_idx)
            print(f"{i}. {anime_name.iloc[0][:40]:40} | Predicted Rating: {predicted_rating:.2f}")

Training Matrix Factorisation Mode
Training matrix factorisation (SVD) with 50 components
SVD model trained
Explained variance ratio: 0.243
SVD Recommendation for User 1:
1. Highschool of the Dead: Drifters of the  | Predicted Rating: 8.15
2. Mahou Shoujo Madoka★Magica               | Predicted Rating: 8.13
3. K-On!                                    | Predicted Rating: 8.10
4. Hellsing                                 | Predicted Rating: 8.09
5. Bishoujo Senshi Sailor Moon              | Predicted Rating: 8.06


## Evaluation

To assess the performance of our recommendation models, we split the dataset into **train** and **test sets**.  
We use a **temporal split**, ensuring that for each user, their most recent ratings form the test set (20% of their ratings).  

We evaluate the models using two types of metrics:

- **Rating Prediction Metrics**
  - **RMSE (Root Mean Squared Error):** Measures the average difference between predicted and actual ratings. Lower is better.
  - **MAE (Mean Absolute Error):** Measures the absolute average difference. Lower is better.

- **Recommendation Quality Metrics**
  - **Precision@K:** Proportion of recommended items that are relevant.
  - **Recall@K:** Proportion of relevant items that were recommended.


In [14]:
def train_test_split_temporal(ratings_df, test_ratio = 0.2):
    # Split the data by time (the last 20% of each users ratings as test)

    train_data = []
    test_data = []

    for user_id in ratings_df['user_id'].unique():
        user_ratings = ratings_df[ratings_df['user_id'] == user_id].sort_values('anime_id')
        n_ratings = len(user_ratings)
        n_test = max(1, int(n_ratings * test_ratio))

        train_data.append(user_ratings.iloc[:-n_test])
        test_data.append(user_ratings.iloc[-n_test:])

    train_df = pd.concat(train_data, ignore_index=True)
    test_df = pd.concat(test_data, ignore_index=True)
    
    return train_df, test_df    

In [15]:
def evaluate_predictions(y_true, y_pred):
    """Calculate RMSE and MAE"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    return rmse, mae

In [16]:
def evalaute_recommendations(model, test_data, user_to_index, item_to_index, k = 10):
    # Evaluate the quality of the recommendations

    precisions = []
    recalls = []

    for user_id in test_data['user_id'].unique():
        if user_id not in user_to_index:
            continue

        user_idx = user_to_index[user_id]
        user_test_items = set(test_data[test_data['user_id'] == user_id]['anime_id'.values])

        try:
            recommendations = model.recommend(user_idx, n_recommendations = k)
            if hasattr(model, 'recommend') and recommendations:
                # Convert the indices to item IDs if needed

                if isinstance(recommendations[0], int) and recommendations[0] < len(item_ids):
                    recommended_items = set(items_ids[idx] for idx in recommendations)
                else:
                    recommended_items = set(recommendations)

                relevent_items = user_test_items.intersection(recommendations)

                precision = len(relevent_items) / len(recommended_items) if recommended_items else 0
                recall = len(relevent_items) / len(user_test_items) if user_test_items else 0

                precisions.append(precision)
                recalls.append(recall)
        
        except:
            continue

    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall
